In [ ]:
from lephare import *
import numpy as np
from matplotlib import pylab as plt

%matplotlib inline

In [ ]:
filter_name = "cosmos/u_cfht.lowres"
f1 = flt(0, filter_name, 1, 0)
f1.read(os.path.join(LEPHAREDIR, "filt", filter_name))
f1.plot_filter_curve()

In [ ]:
filter_driver = Filter(config="../test/COSMOS.para")
filter_driver.run()

In [ ]:
filter_output = os.path.join(
    os.environ["LEPHAREWORK"], "filt", filter_driver.keymap["FILTER_FILE"].value + ".dat"
)
filters = np.loadtxt(filter_output, dtype={"names": ("lamb", "val", "bid"), "formats": (float, float, int)})
plt.loglog(filters["lamb"], filters["val"])
plt.xlabel("wavelength");

# Manipulating a SED object

SED objects represent SED templates belonging to one of the three possible classes "STAR", "QSO" (for AGN type of objects), and "GAL" for galaxies. SED templates available with LePhare can be found under the `sed` directory. 

In [ ]:
!ls $LEPHAREDIR/sed

Let's start with the template of a star. A SED object is created most easily with 3 arguments: its name, an identifying integer, and the type it belongs to. Then calling the `read` function reads the ASCII file passed as argument.

In [ ]:
star_sed = "f2ii.sed"
sed_filename = os.path.join(LEPHAREDIR, "sed/STAR/PICKLES/", star_sed)
sed = StarSED(star_sed, 1, "STAR")
sed.read(sed_filename)

The python code exposes the templates vectors through the `data` method

In [ ]:
x = sed.data()[0]
y = sed.data()[1]
plt.plot(x, y)

In the context of LePhare, such a SED object is going to be written as a byte compressed file, and read back downstream to compute expected magnitudes and to perform the fit. Writing an reading this binary stored file goes as follows:

In [ ]:
rootname = star_sed.split(".")[0]
sed.writeSED(rootname + ".bin", rootname + ".phys", rootname + ".doc")

We can read it back into a new SED object, and check that the values have been correctly read back

In [ ]:
sed2 = StarSED(star_sed, 2, "DONTKNOW")
sed2.readSEDBin(rootname + ".bin")
x2 = sed.data()[0]
y2 = sed.data()[1]
assert np.all(x == x2)
assert np.all(y == y2)
print(sed.jtype, sed2.jtype)

In [ ]:
sed = Sedtolib("../test/COSMOS.para")
sed.run(typ="STAR", star_sed="../test/STAR_MOD_ALL.list", star_fscale="1")

In [ ]:
sed_output = os.path.join(
    os.environ["LEPHAREWORK"], "lib_bin", filter_driver.keymap["STAR_LIB"].value + ".bin"
)

In [ ]:
sed_output

In [ ]:
import struct

buf = open(sed_output, "rb").read()
counter = 0
off = 0
while counter <= 0:
    nrec, jtype, nbw = struct.unpack("iil", buf[off : off + 16])
    wave = struct.unpack("d" * nbw, buf[off + 16 : off + 16 + 8 * nbw])
    print(nrec, jtype, nbw)
    off += 16 + 8 * nbw
    nrec, jtype, nbw = struct.unpack("iil", buf[off : off + 16])
    spec = struct.unpack("d" * nbw, buf[off + 16 : off + 16 + 8 * nbw])
    print(nrec, jtype, nbw)
    off += 16 + 8 * nbw
    counter += 1
plt.plot(wave, spec)
# plt.xlim(1000,1.e4)
# plt.ylim(1.e-10,1.e-6)

In [ ]:
n = 6973
struct.unpack("d", buf[off + 16 + 8 * (n - 1) : off + 16 + 8 * n])

In [ ]:
nbw

In [ ]:
buf[38896 + 16 + 8 : 38896 + 16 + 16]

In [ ]:
struct.unpack("d", buf[38896 + 16 + 8 : 38896 + 16 + 16])

In [ ]:
4.2399775704000006e-08 / 1.23542470e01

In [ ]:
struct.unpack("d" * 1, buf[off + 16 : off + 16 + 8])

In [ ]:
struct.calcsize("l")

In [ ]:
wave[1], spec[1]

In [ ]:
d = np.loadtxt("../sed/STAR/PICKLES/o5v.sed.ext")
plt.plot(d[:, 0], d[:, 1])

In [ ]:
d[:, 1]

In [ ]:
stream_buf = open(sed_output, "rb").read()

In [ ]:
struct.unpack("iil", stream_buf[0:16])

In [ ]:
from lephare import keyword

k = keyword("TEST", "0.1,0.1,6")
print(k.split_double("0.03", 3))
print(k.split_double("0.03", 3))

In [ ]:
mag = MagGal("../test/COSMOS.para")
mag.run(typ="STAR", lib_ascii="YES", star_lib_out="ALLSTAR_COSMOS")

In [ ]:
sed.run(typ="QSO", qso_sed="$LEPHAREDIR/sed/QSO/SALVATO09/AGN_MOD.list")

In [ ]:
sed.run(typ="GAL", gal_sed="../test/COSMOS_MOD.list", gal_lib="LIB_VISTA")

In [ ]:
mag = MagGal("../test/COSMOS.para")
mag.run(
    typ="QSO",
    lib_ascii="NO",
    mod_extinc="0,1000",
    eb_v="0.,0.1,0.2,0.3",
    extinc_law="SB_calzetti.dat",
    z_step="0.04,0,6",
)